# `PyTorch` Multi-Layer Perceptron

We will use the Ionosphere binary (two class) classification dataset to 
demonstrate an MLP for binary classification. This dataset involves predicting 
whether there is a structure in the atmosphere or not given radar returns.

The dataset will be downloaded automatically using Pandas, but you can learn 
more about it here.
- [Ionosphere Dataset (csv)](https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv)
- [Ionosphere Dataset Description](https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.names)

In [1]:
# Imports
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn, optim
from torch.nn import init
import torch

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

### Definitions

In [8]:
class CSVDataset(Dataset):
    """Dataset Definition"""

    def __init__(self, path):
        df = pd.read_csv(path)

        self.X = df.values[:, :-1].astype(np.float32)
        self.y = df.values[:, -1]

        self.y = LabelEncoder().fit_transform(self.y)
        self.y = self.y.astype(np.float32)
        self.y = self.y.reshape((len(self.y), 1))

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return [ self.X[idx], self.y[idx] ]
    
    def get_splits(self, n_test=0.33):
        """ get indexes for train and test rows"""
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])
    

class MLP(nn.Module):
    """MLP Definition"""

    def __init__(self, n_inputs):
        super(MLP, self).__init__()

        # first hidden layer
        self.hidden_1 = nn.Linear(n_inputs, 10)
        init.kaiming_uniform_(self.hidden_1.weight, nonlinearity='relu')
        self.act_1 = nn.ReLU()

        # second hidden layer
        self.hidden_2 = nn.Linear(10, 8)
        init.kaiming_uniform_(self.hidden_2.weight, nonlinearity='relu')
        self.act_2 = nn.ReLU()

        # third hidden layer and output
        self.hidden_3 = nn.Linear(8, 1)
        init.xavier_uniform_(self.hidden_3.weight)
        self.act_3 = nn.Sigmoid()

    def forward(self, X):
        X = self.hidden_1(X)
        X = self.act_1(X)

        X = self.hidden_2(X)
        X = self.act_2(X)

        X = self.hidden_3(X)
        X = self.act_3(X)

        return X
    

def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl


def train_model(train_dl, model):
    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    for epoch in range(100):
        for i, (inputs, labels) in enumerate(train_dl):
            optimizer.zero_grad()
            yhat = model(inputs)
            loss = criterion(yhat, labels)
            loss.backward()
            optimizer.step()


def evaluate_model(test_dl, model):
    pred, actual = list(), list()

    model.eval()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(test_dl):
            yhat = model(inputs)
            yhat = yhat.detach().numpy()
            actual.extend(labels.numpy())
            pred.extend(yhat.round())
    
    pred, actual = np.vstack(pred), np.vstack(actual)

    acc = accuracy_score(actual, pred)
    return acc


def predict(row, model):
    row = torch.tensor([row])
    yhat = model(row)
    yhat = yhat.detach().numpy()
    return yhat

In [4]:
# Prepare the data
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))

234 116


In [10]:
# Define the model
model = MLP(34)

# Train the model
train_model(train_dl, model)

# evaluate the model
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)

Accuracy: 0.905


In [11]:
# make a single prediction (expect class=1)
row = [1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
yhat = predict(row, model)
print('Predicted: %.3f (class=%d)' % (yhat, yhat.round()))

Predicted: 1.000 (class=1)
